In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat
from scipy.optimize import minimize

# Generalized Linear Models (GLMs)

>Question: What are the 3 componenents of a GLM?

*Answer here in **Markdown** and/aor $\LaTeX$*


In previous homework, we worked with retinal spiking data and tried to build a linear encoding model using least squares regression. You may recall the fit of this model was not great. In this problem you will build a Poisson GLM to compare its performance with the linear model.

Let's start by loading the file `rgcData_Uzzel04.mat` into Python, which contains 5 minutes of data from a retinal electrophysiology experiment. You'll find a matrix `stim`, which contains the values of 1D visual stimulus sampled at 120 Hz (the frame rate at which the luminance value of the screen changed) that is 144051 long. The values represent the light level of the screen (relative to a mean level of gray) at each moment in time.  

You'll also find a vector of spike counts `spikes`, which contain the spike counts of a retinal ganglion cell, measured in the same 120Hz time bins as the stimulus (so each entry in the vector is the number of spikes recorded in a 8.33 ms time bin).

This is *real* neural data, first published in [Uzzell & Chichilnisky (2004)](http://jn.physiology.org/content/92/2/780).

In [ ]:

!wget https://github.com/thejuc/314_data/blob/main/314_glm_data.npy?raw=true
data = np.load('314_glm_data.npy?raw=true', allow_pickle=True).item()
data.keys()
stim = data['Stim']
spikes = data['SpikeCounts']


In [ ]:
print('stim shape: ', stim.shape)
print('spikes shape: ', spikes.shape)

Now we want to build a design matrix of the stimulus with 25 time lags. This means the first column of the matrix will be timelag t-24, then t-24,..., until t-0. 

>Construct a design matrix $X$ of size $144051 \times 25$ from the raw stimulus `stim` such that each row (corresponding to a time $t$) contains the 30 most recent values of the stimulus for the current time bin of the response (i.e., the stimulus values at times $t-24, t-23, \ldots , t-1, t-0$). For rows corresponding to stimulus times $t < 24$, simply set values for which there is no data (i.e. $t < 0$) to zero. An easy way to do this might be to start by making a length-144076 vector which contains the stimulus vector `stim` padded with zeros in the first 24 entries. Then you can use a simple `for` loop to build the design matrix.

>Once you construct the timelagged design matrix, add a column of ones to the end of the matrix.

> Visualize the first 50 rows of your design matrix using plt.imshow()

>Let's start by rebuilding our linear model using ordinary least squares regression (OLS). Use your design matrix $X$ and the neuron's responses in `spikes` to compute the least-squares regression solution for the weights (call this weights array `w_ols`).

Hint: We discussed how to compute the least-squares solution for the weights in class. You shouldn't need to use any additional functions beyond what we've already imported for you.

Now let's build our Poisson GLM. There is no closed form solution to this type of GLM like there is for OLS, so instead we have to use an optimizer to find the best weights. We want the optimizer to find the weights which produce the highest likelihood (MLE), $p(Y|X, \omega)$. Since most optimizers find a minima (we want a maxima), we will simply use the negative of the likelihood.

Recall that the probability defined by a poisson distribution is given by: $$p(Y=k) = \frac{\lambda^k e^{-\lambda}}{k!}$$

Where, $k$ is the number of observed spikes and $\lambda$ is the parameter defining the Poisson distribution. In our case, we can ignore the denominator, $k!$, as this just scales our likelihood (not relevant when optimizing).

$\lambda$ in this GLM will be defined as follows:
$$\begin{gather*} \lambda = e^r \\ r = X\vec{\omega} \end{gather*}$$

We will use log-likelihood as this is better for optimization. So, we can redefine our log-likelihood equation as follows (where log is natural-log):
$$\begin{align*} \log(p(Y|X, \omega)) & =  k\log(\lambda) - \lambda \end{align*}$$

This will give us a vector of the log-likelihood of each time point. To get the complete log-likelihood, we just need to take the sum of this vector. Finally, we'll take the negative of this value to get the negative log-likelihood (NLL) of our data.

>Complete the function below to calculate the NLL of our data. Do not change the name of the function or the arguments passed in.


In [ ]:
def nll(w, X, y):
    r = ...   # fill in 
    lam = ... # fill in
    ll = ...  # fill in
    nll = ... # fill in
    return nll

The code cell below will find the optimal weights (`w_glm`) for your GLM given the function you just completed.
>Be sure to fill in X and y with your design matrix and spikes array, respectively

In [ ]:
X = ...   # fill in
y = ...   # fill in
sta = (X.T@y)/y.size
w_glm = minimize(nll, sta.squeeze(), args=(X, y), method='L-BFGS-B').x

>Plot `w_ols` and `w_glm` on the same plot below. Be sure the xticks are correctly labeled for time lags (Hint: each time bin is 1/120 seconds.) Don't plot the last weight in each array (as this corresponds with the ones columns)

>Let's see how the two models compare. Create `y_pred_ols` by matrix multiplying `X @ w_ols`. Next, create `y_pred_glm` by simply calculating $\lambda$ from the equations above (`np.exp(X @ w_glm)`). 
>Now plot the first 120 samples (first second) of the spikes data. On the same plot, add your two predictions. Be sure to add a label to each line and call plt.legend() so that you can see which line corresponds to each model.

>Which model do you think more closely resembles the activity of the neuron? Why do you think this is?

*Answer here in **Markdown** and/aor $\LaTeX$*
